### 1. Upload data for a country (.dta)  (Vietnam)

In [3]:
use "", clear

### 2. Creating ID for the combination of the variables ``FACILITY_ID``, ``DOCTOR_ID``, and ``location_type``
### Keeping only the sum of all ``po_time`` value, and ``location_type`` per ID created

In [4]:
egen id = group(FACILITY_ID DOCTOR_ID location_type)
collapse (sum) po_time (firstnm) location_type , by(id)

variable FACILITY_ID not found


r(111);


### 3. Creating new variables and only keeping necessary variables for the graph

In [5]:
gen study = "Vietnam Commune"
replace study = "Vietnam District" if location_type == 2

gen facilitycode = "V" + string(id)
rename po_time tottime

keep facilitycode study tottime

,sp_case,duration,patients_waiting_in,s4_tr1,s4_tr11,s5_referral,sp_id,pro_age,pro_male,sp1_e1,sp1_e2,sp1_e3,sp1_e5,sp1_e6,sp1_h1,sp1_h2,sp1_h3,sp1_h4,sp1_h5,sp1_h7,sp1_h8,sp1_h9,sp1_h10,sp1_h11,sp1_h12,sp1_h13,sp1_h14,sp1_h15,sp1_h16,sp1_h17,sp1_h18,sp1_h19,sp1_h20,sp1_h21,sp_checklist_items,sp_age,sp_weight,sp_height,sp_male,sp_drugs,sp_drugs_antibio,sp_drugs_quin,sp_drugs_ster,sp_drugs_tb,sp_polypharmacy,n_labs,cxr,sputum,dstgx,correct,q_mbbs,total_price,essential,sp_drugs_antibio_unlab,price_total_usd
1,SP1,6.850000,0,No,Yes,No,4,47,Male,No,Yes,Yes,No,No,Yes,Yes,No,No,No,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,Yes,No,No,No,Yes,12,29,72,154.0,Female,Yes,Yes,No,No,No,4,3,1,0,0,1,1,451.00000,0.500000,1,7.2160
2,SP4,12.800000,0,No,Yes,No,15,47,Male,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,15,42,58,160.0,Male,Yes,No,No,No,No,2,4,1,0,0,0,1,332.30002,0.416667,0,5.3168
3,SP1,6.316667,2,No,Yes,No,3,55,Male,No,Yes,Yes,No,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,7,39,70,153.0,Female,Yes,Yes,Yes,No,No,5,0,0,0,0,0,1,146.00000,0.250000,1,2.3360
4,SP3,1.766667,0,No,Yes,No,13,55,Male,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,4,31,50,160.0,Male,Yes,Yes,Yes,No,No,6,0,0,0,0,0,1,120.00000,0.000000,1,1.9200
5,SP1,5.150000,0,No,No,No,2,48,Male,No,Yes,Yes,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,4,30,74,172.0,Male,Yes,No,No,No,No,3,0,0,0,0,0,1,60.00000,0.250000,1,0.9600
6,SP2,3.066667,2,No,Yes,No,7,48,Male,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,5,38,40,150.0,Female,Yes,No,No,No,No,4,0,0,0,0,0,1,70.00000,0.333333,1,1.1200
7,SP3,10.083333,3,No,Yes,No,11,48,Male,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,7,24,60,168.0,Male,Yes,No,No,No,No,3,1,1,0,0,0,1,70.00000,1.000000,0,1.1200
8,SP2,3.416667,0,Yes,Yes,No,7,48,Male,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,7,38,40,150.0,Female,Yes,Yes,Yes,No,No,6,0,0,0,0,0,1,200.00000,0.166667,1,3.2000
9,SP4,13.833333,0,No,Yes,No,14,48,Male,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,19,42,55,165.5,Male,Yes,Yes,Yes,No,No,6,2,1,1,0,0,1,350.00000,0.666667,1,5.6000
10,SP1,4.233333,1,No,No,No,2,45,Male,No,Yes,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,3,30,74,172.0,Male,Yes,No,No,No,No,6,0,0,0,0,0,0,50.00000,0.250000,1,0.8000


### 4. Temporarily storing the filtered dataset

In [ ]:
tempfile vietnam
save `vietnam' , replace

### 5. Uploading different dataset for different country (.dta) (India)

In [ ]:
use "", clear

### 6. create variable ``tottime``

In [ ]:
gen tottime = .

replace tottime = po_time * po_patients
replace tottime = prov_time * prov_caseload if tottime == .
table study , c(mean tottime count tottime)
keep if tottime != .

### 7. Cleaning data names and keeping only variables necessary for the graph

In [ ]:
replace study = "Birbhum, India" if regexm(study,"Birbhum")
replace study = `""Madhya Pradesh, India" "(Public Clinics)""' if regexm(study,"Mad") & substudy == 2
replace study = `""Madhya Pradesh, India" "(Private Clinics)""' if regexm(study,"Mad") & substudy == 1

collapse (mean) tottime (firstnm) study , by(facilitycode)

### 8. Append the other temporary file and create variable

In [ ]:
append using `vietnam'
gen hours = tottime/60

### 9. Create bar graph

In [ ]:
graph bar hours , $graph_opts1 over(study) ///
	blabel(bar, format(%9.1f)) ///
	hor ylab(1 "1 Hour" 2 "2 Hours" 3 "3 Hours") bar(1,fi(100) lc(black) lw(thin)) ///
	ytit("Average Daily Time Seeing Patients {&rarr}",placement(left) justification(left))

### 10. Export the graph

In [ ]:
graph export "figure.png" , replace width(1000)